# Amplifierd API - Profile Management

This notebook demonstrates profile discovery and management operations.

## Overview

Profiles in amplifier define:
- Session configuration (orchestrator settings)
- Provider configurations (LLM services)
- Tool selections
- Hook configurations
- Agent selections
- Context files

**Profile Schema Versions**:
- **Schema v1** (legacy): Supports one-level inheritance via `extends` field
- **Schema v2** (current): Fully resolved profiles, no runtime inheritance

**Discovery**: Profiles are discovered from collections and cached at:
- `{AMPLIFIERD_HOME}/local/share/profiles/{collection}/{profile}.md`

**Compilation**: Schema v2 profiles can be compiled, resolving all refs:
- Resolves agent, context, and module references
- Creates Python module structure
- Cached at: `{AMPLIFIERD_HOME}/state/profiles/{collection}/{uid}/`

**Activation**: Active profile stored in `active_profile.txt`

This notebook covers both **read operations** and **write operations**.

In [ ]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

## Read Operations

### List All Profiles

Get all available profiles:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/")
profiles = print_response(response, "LIST PROFILES")

if profiles:
    print(f"\n✓ Found {len(profiles)} profile(s)")
    for profile in profiles:
        active = "[ACTIVE]" if profile.get("isActive") else ""
        print(f"  - {profile['name']} {active} (source: {profile['source']})")

### Get Active Profile

Retrieve the currently active profile:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/active")
active_profile = print_response(response, "GET ACTIVE PROFILE")

if active_profile:
    print(f"\n✓ Active profile: {active_profile['name']}")

### Get Profile Details

Get detailed information about a specific profile:

In [ ]:
# Get details for the default profile
profile_name = "default"  # Change this to any available profile

response = requests.get(f"{API_BASE}/profiles/{profile_name}")
profile_details = print_response(response, f"GET PROFILE: {profile_name}")

if profile_details:
    print(f"\n✓ Profile: {profile_details['name']}")
    print(f"  Schema Version: {profile_details.get('schemaVersion', 1)}")
    print(f"  Version: {profile_details.get('version', 'N/A')}")
    print(f"  Description: {profile_details.get('description', 'N/A')}")
    print(f"  Collection: {profile_details.get('collectionId', 'N/A')}")
    print(f"  Source: {profile_details['source']}")
    print(f"  Active: {profile_details['isActive']}")

    # Schema v1: One-level inheritance
    if len(profile_details.get("inheritanceChain", [])) > 1:
        base = profile_details["inheritanceChain"][1]
        print(f"  Extends: {base}")

    # Schema v2: No inheritance at runtime
    if profile_details.get("schemaVersion", 1) == 2:
        print("  Schema v2: Profile is fully resolved (no runtime inheritance)")

    print(f"  Providers: {len(profile_details.get('providers', []))}")
    print(f"  Tools: {len(profile_details.get('tools', []))}")
    print(f"  Hooks: {len(profile_details.get('hooks', []))}")

### Explore Profile Components

Examine providers, tools, and hooks in a profile:

In [ ]:
if profile_details:
    print("Providers:")
    for provider in profile_details.get("providers", []):
        module_ref = provider["module"]
        print(f"  - {module_ref}")

    print("\nTools:")
    for tool in profile_details.get("tools", []):
        print(f"  - {tool['module']}")

    print("\nHooks:")
    for hook in profile_details.get("hooks", []):
        print(f"  - {hook['module']}")

## Write Operations

### Activate a Profile (Schema v1)

Set a schema v1 profile as active:

In [ ]:
# Activate a schema v1 profile (use a profile name from the list above)
profile_to_activate = "default"

response = requests.post(f"{API_BASE}/profiles/{profile_to_activate}/activate")
result = print_response(response, f"ACTIVATE PROFILE (v1): {profile_to_activate}")

if response.ok:
    print(f"\n✓ Activated profile: {result['name']}")
    print(f"  Status: {result['status']}")
elif response.status_code == 404:
    print(f"\n✗ Profile '{profile_to_activate}' not found")
else:
    print(f"\n✗ Failed to activate profile: {response.status_code}")

In [ ]:
# Compile and activate a schema v2 profile
collection_id = "mycoll"  # Collection containing the profile
profile_name = "myprofile"  # Schema v2 profile name

response = requests.post(f"{API_BASE}/profiles/{collection_id}/{profile_name}/compile")
result = print_response(response, f"COMPILE PROFILE: {collection_id}/{profile_name}")

if response.ok:
    print("\n✓ Profile compiled and activated")
    print(f"  Collection: {result['collection_id']}")
    print(f"  Profile: {result['profile_name']}")
    print(f"  Compiled at: {result['compiled_path']}")
    print(f"  Status: {result['status']}")
elif response.status_code == 404:
    print(f"\n✗ Profile not found: {collection_id}/{profile_name}")
elif response.status_code == 400:
    print(f"\n✗ Compilation error: {result.get('detail', 'Unknown error')}")
else:
    print(f"\n✗ Failed to compile profile: {response.status_code}")

### Compile and Activate Profile (Schema v2)

Schema v2 profiles support compilation which resolves all refs and creates a Python module structure:

### Verify Profile Activation

Confirm the profile is now active:

In [ ]:
response = requests.get(f"{API_BASE}/profiles/active")
active = print_response(response, "VERIFY ACTIVATION")

if active:
    print(f"\n✓ Confirmed: '{active['name']}' is active")

### Deactivate Current Profile

Clear the active profile:

In [ ]:
response = requests.delete(f"{API_BASE}/profiles/active")
result = print_response(response, "DEACTIVATE PROFILE")

if response.ok:
    print("\n✓ Profile deactivated")

    # Verify no active profile
    response = requests.get(f"{API_BASE}/profiles/active")
    if response.status_code == 200:
        active = response.json()
        if not active:
            print("✓ Confirmed: No active profile")
    else:
        print("✓ Confirmed: No active profile")

### Error Handling: Non-Existent Profile

Attempt to activate a profile that doesn't exist:

In [ ]:
fake_profile = "non-existent-profile"
response = requests.post(f"{API_BASE}/profiles/{fake_profile}/activate")
print_response(response, f"ACTIVATE NON-EXISTENT: {fake_profile}")

if response.status_code == 404:
    print("\n✓ Correctly returns 404 for non-existent profile")

## Complete Profile Workflow

Demonstrate a full profile management workflow:

In [ ]:
def profile_workflow():
    """Complete profile management workflow."""

    # 1. List available profiles
    print("1. Listing profiles...")
    response = requests.get(f"{API_BASE}/profiles/")
    if not response.ok:
        print("✗ Failed to list profiles")
        return

    profiles = response.json()
    print(f"✓ Found {len(profiles)} profiles")

    # 2. Get details for first profile
    if profiles:
        profile_name = profiles[0]["name"]
        print(f"\n2. Getting details for '{profile_name}'...")
        response = requests.get(f"{API_BASE}/profiles/{profile_name}")
        if response.ok:
            details = response.json()
            print(f"✓ Profile has {len(details.get('providers', []))} providers")

        # 3. Activate the profile
        print(f"\n3. Activating '{profile_name}'...")
        response = requests.post(f"{API_BASE}/profiles/{profile_name}/activate")
        if response.ok:
            print("✓ Activated successfully")

        # 4. Verify activation
        print("\n4. Verifying activation...")
        response = requests.get(f"{API_BASE}/profiles/active")
        if response.ok:
            active = response.json()
            if active and active["name"] == profile_name:
                print(f"✓ Confirmed: '{profile_name}' is active")

    print("\n✓ Workflow complete")


profile_workflow()

## Schema v1 → v2 Migration

**Schema v1 (Legacy)**:
- Supports `extends` field for inheritance
- Still works but deprecated
- Cannot use compilation features

**Schema v2 (Current)**:
- No `extends` field (manually resolve inheritance)
- Required `schema-version: 2` field
- Supports compilation with ref resolution
- Recommended for all new profiles

**Migration Steps**:
1. Resolve any `extends` references manually
2. Add `schema-version: 2` to profile section
3. Update agent refs to individual files
4. Update context refs to directories
5. Test profile loads correctly

**Example Migration**:

Schema v1:
```yaml
profile:
  name: myprofile
  extends: base-profile
  
providers:
  - module: openai
```

Schema v2:
```yaml
profile:
  name: myprofile
  schema-version: 2  # Add this!
  # No extends - manually merge from base-profile

providers:
  - module: openai
    source: git+https://github.com/org/modules@main
```

## Summary

### Read Operations
- ✓ List all available profiles
- ✓ Get active profile
- ✓ Get detailed profile information (including schema version)
- ✓ Explore profile components (providers, tools, hooks, agents, context)
- ✓ Schema v1: One-level inheritance via `extends` field
- ✓ Schema v2: Fully resolved profiles with compilation support

### Write Operations
- ✓ Activate a profile (schema v1)
- ✓ Compile and activate a profile (schema v2)
- ✓ Deactivate current profile
- ✓ Error handling for non-existent profiles

## Profile Discovery

**Directory Structure**:
```
{AMPLIFIERD_HOME}/local/share/
└── profiles/
    └── {collection}/
        ├── default.md          # Schema v2 profile
        └── advanced.md         # Schema v2 profile
```

**Discovery Process**:
- Profiles discovered from installed collections
- Cached in `profiles/{collection}/{profile}.md`
- Schema version detected from profile content

## Profile Formats

### Schema v1 (Legacy)
```yaml
profile:
  name: my-profile
  version: "1.0.0"
  extends: base-profile  # Optional one-level inheritance

providers:
  - module: openai
```

### Schema v2 (Current)
```yaml
---
profile:
  name: my-profile
  schema-version: 2      # Required!
  version: "1.0.0"

session:
  orchestrator:
    module: loop-streaming
    source: git+https://github.com/org/modules@main

agents:
  - ref: git+https://github.com/org/repo@main/agents/agent.md

context:
  - ref: git+https://github.com/org/repo@main/context/
---

# Profile description
```

## API Endpoints Reference

| Method | Endpoint | Description | Schema |
|--------|----------|-------------|--------|
| GET | `/api/v1/profiles/` | List all profiles | v1, v2 |
| GET | `/api/v1/profiles/active` | Get active profile | v1, v2 |
| GET | `/api/v1/profiles/{name}` | Get profile details | v1, v2 |
| POST | `/api/v1/profiles/{name}/activate` | Activate profile (v1) | v1 |
| POST | `/api/v1/profiles/{coll}/{name}/compile` | Compile & activate (v2) | v2 |
| DELETE | `/api/v1/profiles/active` | Deactivate profile | v1, v2 |

## Profile Compilation (Schema v2 Only)

**Compilation Process**:
1. Resolve all refs (agents, context, modules)
2. Fetch and cache resolved assets
3. Create Python module structure
4. Store in `state/profiles/{collection}/{uid}/`

**Benefits**:
- Fast startup (pre-fetched assets)
- Explicit dependencies
- Version control
- Dynamic module loading

## Next Steps

Continue to:
- **04-collection-management.ipynb** - Collection package management
- **05-module-management.ipynb** - Module discovery

## Profile Schema v2 Format

Schema v2 profiles are the current standard and support compilation:

```yaml
---
profile:
  name: myprofile
  schema-version: 2            # Required!
  version: 1.0.0
  description: "Production profile"

session:
  orchestrator:
    module: loop-streaming
    source: git+https://github.com/org/amplifier-module-loop-streaming@main
    config:
      extended_thinking: true

tools:
  - module: tool-web
    source: git+https://github.com/org/amplifier-module-tool-web@main

agents:
  - ref: git+https://github.com/org/repo@main/agents/researcher.md
  - ref: /absolute/path/to/agent.md

context:
  - ref: git+https://github.com/org/repo@main/context/
---

# Profile description (markdown content)
```

**Key Differences from Schema v1**:
- `schema-version: 2` required
- No `extends` field (profiles fully resolved at authoring time)
- Agents as individual file refs
- Context as directory refs
- Support for compilation

In [ ]:
# Example: Sync modules for active profile
if profile_details:
    profile_name = profile_details["name"]

    # Get the active profile's required modules and sync them
    response = requests.post(f"{API_BASE}/profiles/sync-modules")
    result = print_response(response, f"SYNC MODULES FOR PROFILE: {profile_name}")

    if response.ok:
        print("\n✓ Modules synced successfully")
        print(f"  Modules installed: {result.get('modulesSynced', 0)}")
        if "modules" in result:
            print("\n  Module details:")
            for module in result["modules"][:5]:  # Show first 5
                print(f"    - {module['id']}")
                print(f"      Status: {module.get('status', 'unknown')}")
                print(f"      Cache key: {module.get('cacheKey', 'N/A')[:16]}...")
    else:
        print(f"\n✗ Failed to sync modules: {result.get('status', 'unknown error')}")
        if "failures" in result:
            print("\n  Failures:")
            for failure in result["failures"]:
                print(f"    - Source: {failure['source']}")
                print(f"      Error: {failure['error']}")
else:
    print("No active profile. Activate a profile first to sync its modules.")

### API Endpoints for Module Sync

| Method | Endpoint | Description |
|--------|----------|-------------|
| POST | `/api/v1/profiles/sync-modules` | Sync modules for active profile |
| GET | `/api/v1/profiles/{name}/module-sources` | Get module sources for profile |

### Key Concepts: Profile-Driven Module Resolution

**Module Dependency Declaration**:
- Profiles specify `modules.sources` with git URLs
- Each source declares which specific modules to install
- Multiple sources can be combined in one profile

**Automatic Installation**:
- Triggered by profile sync operation
- Modules cloned/pulled from declared sources
- Cached using git commit hashes (content-addressable)
- Symlinks created for deduplication

**Version Management**:
- Multiple module versions coexist in cache
- Active version accessible via symlinks
- Easy rollback to previous versions
- Unused versions can be cleaned up

## Profile-Driven Module Resolution

Profiles can specify module sources and dependencies that are automatically resolved when the profile is synchronized. This enables profiles to describe their complete configuration including module installations.

### Module Sources in Profiles

Profiles can declare module sources that should be installed along with the profile:

```yaml
profile:
  name: my-profile
  version: "1.0.0"

modules:
  sources:
    - url: https://github.com/my-org/modules.git
      branch: main
      directory: modules/
      modules:
        - my-collection/provider/custom-llm
        - my-collection/tool/advanced-search

providers:
  - module: my-collection/provider/custom-llm
    config:
      api_key: "${env:CUSTOM_LLM_KEY}"

tools:
  - module: my-collection/tool/advanced-search
```

### Profile Sync with Module Installation

When a profile is synced, all specified modules are automatically installed:

```python
import requests

# Sync a profile - includes module installation
response = requests.post(f"{API_BASE}/profiles/sync/{profile_name}")
result = response.json()

# Result includes module installation status
print(f"Profile synced: {result['status']}")
print(f"Modules installed: {result['modulesInstalled']}")
```

### Modules Endpoint for Profile Sync

Manually trigger module installation for a profile:

```python
# Sync modules for the active profile
response = requests.post(f"{API_BASE}/profiles/sync-modules")

# Result
{
  "status": "success",
  "profileName": "my-profile",
  "modulesSynced": 3,
  "modules": [
    {
      "id": "my-collection/provider/custom-llm",
      "status": "installed",
      "cacheKey": "abc123def456"
    },
    {
      "id": "my-collection/tool/advanced-search",
      "status": "installed",
      "cacheKey": "xyz789"
    }
  ]
}
```

### Module Caching and Deduplication

Modules are cached using content-addressable storage based on git commit hashes:

```
{AMPLIFIERD_HOME}/local/share/modules/
├── _cache/
│   └── {git_hash}/
│       └── my-collection/
│           ├── providers/openai/module.yaml
│           ├── tools/search/module.yaml
│           └── hooks/logging/module.yaml
├── my-collection/  # Symlinks to latest version
│   ├── providers/
│   │   └── openai/ -> ../../_cache/{current_hash}/my-collection/providers/openai
│   └── tools/
│       └── search/ -> ../../_cache/{current_hash}/my-collection/tools/search
```

**Benefits**:
- **Deduplication**: Same module version symlinked, not duplicated
- **Version management**: Multiple versions coexist in cache
- **Fast installation**: Skip re-downloading unchanged modules
- **Clean removal**: Remove unused cached versions

### Module Installation State Cache

Profile module state is cached for quick lookup:

```python
# Cache structure
{AMPLIFIERD_HOME}/.cache/
└── module_state.json

# Example content
{
  "profile": "my-profile",
  "lastSync": "2025-11-21T10:30:00Z",
  "modules": {
    "my-collection/provider/custom-llm": {
      "version": "1.0.0",
      "cacheKey": "abc123def456",
      "installed": true,
      "path": "/path/to/.../modules/my-collection/providers/custom-llm"
    }
  }
}
```

### Error Handling for Module Installation

Module installation failures are reported clearly:

```python
# Attempt to sync profile with unavailable module source
response = requests.post(f"{API_BASE}/profiles/sync-modules")

# Error response
{
  "status": "partial_failure",
  "profileName": "my-profile",
  "modulesSynced": 2,
  "failures": [
    {
      "source": "https://github.com/unavailable/repo.git",
      "error": "Repository not found",
      "modules": ["my-collection/provider/unavailable"],
      "recommendation": "Check source URL or authentication"
    }
  ]
}
```

## Summary

### Read Operations
- ✓ List all available profiles from flattened directory structure
- ✓ Get active profile
- ✓ Get detailed profile information
- ✓ Explore profile components (providers, tools, hooks)
- ✓ One-level inheritance via `extends` field

### Write Operations
- ✓ Activate a profile (writes to `active_profile.txt`)
- ✓ Deactivate current profile (removes `active_profile.txt`)
- ✓ Error handling for non-existent profiles

## Profile Discovery

**Flattened Directory Structure**:
```
{AMPLIFIERD_HOME}/local/share/
└── profiles/
    ├── {collection}/           # Profiles from installed collections
    │   ├── default.yaml
    │   └── advanced.yaml
    └── my-profile.yaml         # Standalone profile (no collection)
```

**Discovery Process**:
- Scans `profiles/**/*.yaml` recursively
- Both collection and standalone profiles discovered together
- User sees "profiles" not "collections with profiles"
- Like `/etc/profile.d/` in Linux

## Profile Format

Profiles are YAML files located in the flattened `profiles/` directory:

```yaml
profile:
  name: my-profile
  version: "1.0.0"
  description: "Profile description"
  extends: base-profile  # Optional one-level inheritance

providers:
  - module: collection/provider/openai
    config:
      model: gpt-4

tools:
  - module: collection/tool/web-search

hooks:
  - module: collection/hook/logging
```

**Creating Standalone Profiles**:
Simply place a YAML file in `{AMPLIFIERD_HOME}/local/share/profiles/` - no collection required!

## API Endpoints Reference

| Method | Endpoint | Description | Type |
|--------|----------|-------------|-------|
| GET | `/api/v1/profiles/` | List all profiles | Read |
| GET | `/api/v1/profiles/active` | Get active profile | Read |
| GET | `/api/v1/profiles/{name}` | Get profile details | Read |
| POST | `/api/v1/profiles/{name}/activate` | Activate profile | Write |
| DELETE | `/api/v1/profiles/active` | Deactivate profile | Write |

## Next Steps

Continue to:
- **04-collection-management.ipynb** - Collection package management
- **05-module-management.ipynb** - Module discovery